# Step 2 - Test The Model



In [1]:
from keras.models import load_model
import sys
import numpy as np
import glob
import os

if ('../PythonClient/' not in sys.path):
    sys.path.insert(0, '../PythonClient/')
from AirSimClient import *

# GPU Memory Error 
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# << Set this to the path of the model >>
# If None, then the model with the lowest validation loss from training will be used
MODEL_PATH = None

if (MODEL_PATH == None):
    models = glob.glob('model/models/*.h5') 
    best_model = max(models, key=os.path.getctime)
    MODEL_PATH = best_model
    
print('Using model {0} for testing.'.format(MODEL_PATH))

Using TensorFlow backend.


Using model model/models\model_model.24-0.0097591.h5 for testing.


모델을 로드하고, Landscape 환경에서 Airsim Simulator와 연결한다. 이 단계에서는 반드시 시뮬레이터가 실행되고 있어야 한다.

In [2]:
model = load_model(MODEL_PATH)

client = CarClient()
client.confirmConnection()
client.enableApiControl(True)
car_controls = CarControls()
print('Connection established!')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


TransportError: Retry connection over the limit

모델의 출력을 젖아하는 데, 사용되는 버퍼 뿐만 아니라 자동차의 초기 상태를 설정

In [3]:
car_controls.steering = 0
car_controls.throttle = 0
car_controls.brake = 0

image_buf = np.zeros((1, 59, 255, 3))
state_buf = np.zeros((1,4))

모델에서 사용할 RGB 이미지를 읽기 위한 기능을 정의한다.


In [4]:
def get_image():
    image_response = client.simGetImages([ImageRequest(0, AirSimImageType.Scene, False, False)])[0]
    image1d = np.fromstring(image_response.image_data_uint8, dtype=np.uint8)
    image_rgba = image1d.reshape(image_response.height, image_response.width, 4)
    
    return image_rgba[76:135,0:255,0:3].astype(float)

해당 모델은 속도를 예측할 수 없기 때문에, 10m/s의 일정한 속도로 주행하고, 모델을 통해 조향 각도를 조정하게 된다.

In [5]:
while (True):
    car_state = client.getCarState()
    
    if (car_state.speed < 10):
        car_controls.throttle = 1
    else:
        car_controls.throttle = 0.0
    
    image_buf[0] = get_image()
    state_buf[0] = np.array([car_controls.steering, car_controls.throttle, car_controls.brake, car_state.speed])
    model_output = model.predict([image_buf, state_buf])
    car_controls.steering = round(0.5 * float(model_output[0][0]), 2)
    
    print('Sending steering = {0}, throttle = {1}'.format(car_controls.steering, car_controls.throttle))
    
    client.setCarControls(car_controls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until


Sending steering = -0.33, throttle = 1
Sending steering = -0.52, throttle = 1
Sending steering = -0.35, throttle = 1
Sending steering = -0.29, throttle = 1
Sending steering = -0.28, throttle = 1
Sending steering = -0.21, throttle = 1
Sending steering = -0.19, throttle = 1
Sending steering = -0.28, throttle = 1
Sending steering = -0.22, throttle = 1
Sending steering = -0.23, throttle = 1
Sending steering = -0.47, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.11, throttle = 1
Sending steering = -0.16, throttle = 1
Sending steering = -0.56, throttle = 1
Sending steering = -0.14, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.1, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.01, throttle = 1
Sending steering = -0.27, throttle = 1
Sending steering = -0.38, throttle = 1
Sending steering = -0.34, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.04, throt

Sending steering = 0.13, throttle = 0.0
Sending steering = 0.02, throttle = 0.0
Sending steering = -0.22, throttle = 0.0
Sending steering = -0.47, throttle = 1
Sending steering = -1.28, throttle = 1
Sending steering = -0.59, throttle = 1
Sending steering = -1.1, throttle = 1
Sending steering = -0.8, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.9, throttle = 1
Sending steering = -0.55, throttle = 1
Sending steering = -0.63, throttle = 1
Sending steering = 0.13, throttle = 1
Sending steering = -0.62, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.18, throttle = 1
Sending steering = -0.19, throttle = 1
Sending steering = -1.63, throttle = 1
Sending steering = -0.65, throttle = 1
Sending steering = -0.06, throttle = 1
Sending steering = -0.37, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -1.21, throttle = 1
Sending steering = -0.87, throttle = 1
Sending steering = -0.65, thr

Sending steering = -0.19, throttle = 1
Sending steering = -0.16, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.03, throttle = 0.0
Sending steering = 0.05, throttle = 0.0
Sending steering = 0.17, throttle = 0.0
Sending steering = 0.23, throttle = 0.0
Sending steering = 0.56, throttle = 0.0
Sending steering = 0.38, throttle = 0.0
Sending steering = 0.4, throttle = 0.0
Sending steering = 0.17, throttle = 0.0
Sending steering = 0.21, throttle = 0.0
Sending steering = 0.1, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.04, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.45, 

Sending steering = -0.21, throttle = 0.0
Sending steering = -0.44, throttle = 1
Sending steering = -0.56, throttle = 1
Sending steering = -0.69, throttle = 1
Sending steering = -0.58, throttle = 1
Sending steering = -0.36, throttle = 1
Sending steering = -0.03, throttle = 1
Sending steering = 0.25, throttle = 1
Sending steering = 0.18, throttle = 1
Sending steering = 0.32, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.14, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.04, throttle = 1
Sending steering = -0.05, throttle = 1
Sending steering = -0.01, throttle = 1
Sending steering = 0.24, throttle = 1
Sending steering = 0.26, throttle = 1
Sending steering = 0.06, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.4, throttle = 1
Sending steering = -0.11, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1

...

Sending steering = -0.02, throttle = 1
Sending steering = -0.01, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = -0.27, throttle = 0.0
Sending steering = -0.43, throttle = 0.0
Sending steering = -0.18, throttle = 0.0
Sending steering = -0.08, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.21, throttle = 0.0
Sending steering = 0.23, throttle = 0.0
Sending steering = 0.54, throttle = 0.0
Sending steering = 0.49, throttle = 0.0
Sending steering = 0.33, throttle = 0.0
Sending steering = 0.14, throttle = 0.0
Sending steering = 0.15, throttle = 1
Sending steering = -0.23, throttle = 1
Sending steering = -0.34, throttle = 1
Sending steering = -0.52, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.16, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.37, throttle = 1
Sending steering = 0.02, throttle = 1
Sending steering = 0.1, throttle = 1
Sending steering = 0.14, throttle = 1
Sending steering

Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.08, throttle = 1
Sending steering = 0.07, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.04, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.14, throttle = 1
Sending steering = -0.31, throttle = 1
Sending steering = -0.17, throttle = 1
Sending steering = -0.22, throttle = 1
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.21, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.12, throttle = 0.0
Sending steering = 1.45, throttle = 0.0
Sending steering = 1.06, thrott

Sending steering = -0.17, throttle = 1
Sending steering = -0.52, throttle = 1
Sending steering = -0.12, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = -0.04, throttle = 1
Sending steering = -0.15, throttle = 1
Sending steering = -0.51, throttle = 1
Sending steering = -0.39, throttle = 1
Sending steering = -0.54, throttle = 1
Sending steering = -0.19, throttle = 1
Sending steering = 0.01, throttle = 1
Sending steering = 0.09, throttle = 0.0
Sending steering = 0.34, throttle = 0.0
Sending steering = 0.04, throttle = 0.0
Sending steering = 0.01, throttle = 0.0
Sending steering = 0.01, throttle

Sending steering = -0.1, throttle = 0.0
Sending steering = 0.02, throttle = 0.0
Sending steering = 0.65, throttle = 0.0
Sending steering = 0.74, throttle = 0.0
Sending steering = 0.43, throttle = 0.0
Sending steering = 0.28, throttle = 0.0
Sending steering = -0.12, throttle = 1
Sending steering = -0.05, throttle = 1
Sending steering = -1.01, throttle = 1
Sending steering = -0.66, throttle = 1
Sending steering = -0.85, throttle = 1
Sending steering = -1.04, throttle = 1
Sending steering = -0.73, throttle = 1
Sending steering = 0.09, throttle = 1
Sending steering = 0.42, throttle = 1
Sending steering = 1.08, throttle = 1
Sending steering = 3.2, throttle = 1
Sending steering = 4.19, throttle = 1
Sending steering = 3.22, throttle = 1
Sending steering = 3.11, throttle = 1
Sending steering = 1.17, throttle = 1
Sending steering = 0.4, throttle = 1
Sending steering = -1.11, throttle = 1
Sending steering = -1.58, throttle = 1
Sending steering = -1.12, throttle = 1
Sending steering = -0.99, thro

KeyboardInterrupt: 